In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import geopandas as gpd

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executer.memory", "4g")
    .getOrCreate()
)

22/08/30 15:33:18 WARN Utils: Your hostname, Xiaotongs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.16.205 instead (on interface en0)
22/08/30 15:33:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/08/30 15:33:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/30 15:33:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
merchant_df = pd.read_parquet("../data/tables/tbl_merchants.parquet")
print("Number of merchants in the dataset:", merchant_df.shape[0])
print("Columns in merchant dataset:", [merchant_df.index.name] + merchant_df.columns.tolist())

# extract tags, revenue level and take rate from the "tags" column
merchant_df["tags"] = merchant_df["tags"].str.findall(r'[\(\[]+([^\)\]]*)[\)\]]')
merchant_df["revenue_level"] = merchant_df["tags"].str[1]
merchant_df["take_rate"] = merchant_df["tags"].str[2].str.extract(r'[^\d]*([\d.]*)').astype(float)
merchant_df["tags"] = merchant_df["tags"].str[0].str.lower()  # convert all letters to lowercase
merchant_df

Number of merchants in the dataset: 4026
Columns in merchant dataset: ['merchant_abn', 'name', 'tags']


,name,tags,revenue_level,take_rate
merchant_abn,,,,
10023283211,Felis Limited,"furniture, home furnishings and equipment shop...",e,0.18
10142254217,Arcu Ac Orci Corporation,"cable, satellite, and other pay television and...",b,4.22
10165489824,Nunc Sed Company,"jewelry, watch, clock, and silverware shops",b,4.40
10187291046,Ultricies Dignissim Lacus Foundation,"watch, clock, and jewelry repair shops",b,3.29
10192359162,Enim Condimentum PC,"music shops - musical instruments, pianos, and...",a,6.33
...,...,...,...,...
99938978285,Elit Dictum Eu Ltd,"opticians, optical goods, and eyeglasses",b,4.50
99974311662,Mollis LLP,"books, periodicals, and newspapers",b,3.17
99976658299,Sociosqu Corp.,shoe shops,a,6.57


In [4]:
transaction_sdf = spark.read.parquet("../data/tables/transactions_20210228_20210827_snapshot/")
transaction_sdf.count()

3643266

In [5]:
transaction_sdf.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [6]:
transaction_sdf

user_id,merchant_abn,dollar_value,order_id,order_datetime
18478,62191208634,63.255848959735246,949a63c8-29f7-4ab...,2021-08-20
2,15549624934,130.3505283105634,6a84c3cf-612a-457...,2021-08-20
18479,64403598239,120.15860593212783,b10dcc33-e53f-425...,2021-08-20
3,60956456424,136.6785200286976,0f09c5a5-784e-447...,2021-08-20
18479,94493496784,72.96316578355305,f6c78c1a-4600-4c5...,2021-08-20
3,76819856970,448.529684285612,5ace6a24-cdf0-4aa...,2021-08-20
18479,67609108741,86.4040605836911,d0e180f0-cb06-42a...,2021-08-20
3,34096466752,301.5793450525113,6fb1ff48-24bb-4f9...,2021-08-20
18482,70501974849,68.75486276223054,8505fb33-b69a-412...,2021-08-20
4,49891706470,48.89796461900801,ed11e477-b09f-4ae...,2021-08-20


In [85]:
consumer_sdf = spark.read.option("delimiter", "|").csv("../data/tables/tbl_consumer.csv", inferSchema =True, header=True)
id_sdf = spark.read.parquet("../data/tables/consumer_user_details.parquet")
for field in ("consumer", "user"):
    field = f'{field}_id'
    id_sdf = id_sdf.withColumn(field, F.col(field).cast('INT'))
output = id_sdf.join(consumer_sdf,["consumer_id"],how='outer')
output.printSchema()

root
 |-- consumer_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)



In [78]:
# map user id to consumer id
consumer_df = pd.read_csv("../data/tables/tbl_consumer.csv", delimiter="|")
consumer_user_df = pd.read_parquet("../data/tables/consumer_user_details.parquet")
consumer = consumer_user_df.merge(consumer_df, how="outer", on="consumer_id")
consumer

,user_id,consumer_id,name,address,state,postcode,gender
0,1,1195503,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female
1,2,179208,Mary Smith,3764 Amber Oval,NSW,2782,Female
2,3,1194530,Jill Jones MD,40693 Henry Greens,NT,862,Female
3,4,154128,Lindsay Jimenez,00653 Davenport Crossroad,NSW,2780,Female
4,5,712975,Rebecca Blanchard,9271 Michael Manors Suite 651,WA,6355,Female
...,...,...,...,...,...,...,...
499994,499995,1385608,Jessica Avila,508 Miranda Overpass Apt. 218,QLD,4400,Female
499995,499996,1466964,Steven Thornton,7913 Schwartz Mission Suite 483,VIC,3097,Undisclosed
499996,499997,1253484,Christy Smith,5681 Zachary Mountain Apt. 060,NSW,2756,Undisclosed
499997,499998,175005,Donna Sutton,54140 Jacob Point,VIC,3989,Female


In [8]:
consumer.groupby("postcode")["postcode"].count().reset_index(name="count")

,postcode,count
0,200,145
1,800,155
2,801,147
3,804,156
4,810,150
...,...,...
3162,9013,137
3163,9015,136
3164,9464,165
3165,9726,178


In [59]:
sf = gpd.read_file("../data/curated/SA2_2021/SA2_2021_AUST_GDA2020.shp")
sf = sf.set_index('SA2_CODE21')
sf = sf.loc[sf.geometry != None]
sf

,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
SA2_CODE21,,,,,,,,,,,,,,,,
101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801111141,Namadgi,0,No change,80111,Uriarra - Namadgi,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,1202.7527,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((148.80407 -35.37619, 148.80417 -35.3..."
901011001,Christmas Island,0,No change,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,136.1356,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((105.67393 -10.41566, 105.67399 -10.4..."
901021002,Cocos (Keeling) Islands,0,No change,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,13.7258,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((96.91512 -12.14044, 96.91513 -..."


In [16]:
# sf_poa = gpd.read_file("../data/curated/POA_2021/POA_2021_AUST_GDA2020.shp")

# sf_poa = sf_poa.loc[sf_poa["geometry"] != None]
# sf_poa['postcode'] = sf_poa['POA_CODE21'].astype(int)
# sf_poa.head()

In [86]:
# link: https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv
all_postcode = pd.read_csv("../data/curated/australian_postcodes.csv")
all_postcode.columns

Index(['id', 'postcode', 'locality', 'state', 'long', 'lat', 'dc', 'type',
       'status', 'sa3', 'sa3name', 'sa4', 'sa4name', 'region', 'Lat_precise',
       'Long_precise', 'SA1_MAINCODE_2011', 'SA1_MAINCODE_2016',
       'SA2_MAINCODE_2016', 'SA2_NAME_2016', 'SA3_CODE_2016', 'SA3_NAME_2016',
       'SA4_CODE_2016', 'SA4_NAME_2016', 'RA_2011', 'RA_2016', 'MMM_2015',
       'MMM_2019', 'ced', 'altitude', 'chargezone', 'phn_code', 'phn_name',
       'lgaregion', 'electorate', 'electoraterating'],
      dtype='object')

In [87]:
all_postcode = all_postcode[['postcode', 'long', 'lat']]
all_postcode

,postcode,long,lat
0,200,149.119000,-35.277700
1,200,149.118900,-35.277700
2,800,130.836680,-12.458684
3,800,130.836680,-12.458684
4,801,130.836680,-12.458684
...,...,...,...
18437,9013,152.823141,-27.603479
18438,9015,152.823141,-27.603479
18439,9464,153.074982,-27.397055
18440,9726,153.412197,-28.008783


In [88]:
all_postcode = all_postcode.drop_duplicates(subset='postcode', keep="first")
all_postcode

,postcode,long,lat
0,200,149.119000,-35.277700
2,800,130.836680,-12.458684
4,801,130.836680,-12.458684
5,804,130.873315,-12.428017
6,810,130.866242,-12.381806
...,...,...,...
18437,9013,152.823141,-27.603479
18438,9015,152.823141,-27.603479
18439,9464,153.074982,-27.397055
18440,9726,153.412197,-28.008783


In [89]:
from shapely.geometry import Point, Polygon

all_postcode["coordinate"] = all_postcode.apply(lambda x: Point(x.long, x.lat), axis=1)
all_postcode

/Users/chenluyao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:112: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  values = construct_1d_object_array_from_listlike(values)
<ipython-input-89-63b530203a2f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_postcode["coordinate"] = all_postcode.apply(lambda x: Point(x.long, x.lat), axis=1)


,postcode,long,lat,coordinate
0,200,149.119000,-35.277700,POINT (149.119 -35.2777)
2,800,130.836680,-12.458684,POINT (130.83668 -12.458684)
4,801,130.836680,-12.458684,POINT (130.83668 -12.458684)
5,804,130.873315,-12.428017,POINT (130.873315 -12.428017)
6,810,130.866242,-12.381806,POINT (130.866242 -12.381806)
...,...,...,...,...
18437,9013,152.823141,-27.603479,POINT (152.823141 -27.603479)
18438,9015,152.823141,-27.603479,POINT (152.823141 -27.603479)
18439,9464,153.074982,-27.397055,POINT (153.074982 -27.397055)
18440,9726,153.412197,-28.008783,POINT (153.412197 -28.008783)


In [60]:
def postcode_to_SA2(coordinate):
    for SA2_code, row in sf.iterrows():
        if coordinate.within(row['geometry']):
            return SA2_code
            
all_postcode["SA2_code"] = all_postcode.apply(lambda x: postcode_to_SA2(x.coordinate), axis=1)
all_postcode

,postcode,long,lat,Long_precise,Lat_precise,coordinate,SA2_code
0,200,149.119000,-35.277700,149.119000,-35.277700,POINT (149.119 -35.2777),801051049
2,800,130.836680,-12.458684,130.776661,-12.393279,POINT (130.83668 -12.458684),701011002
4,801,130.836680,-12.458684,130.845642,-12.463440,POINT (130.83668 -12.458684),701011002
5,804,130.873315,-12.428017,130.846254,-12.432480,POINT (130.873315 -12.428017),701011009
6,810,130.866242,-12.381806,130.873000,-12.380000,POINT (130.866242 -12.381806),701021010
...,...,...,...,...,...,...,...
18437,9013,152.823141,-27.603479,153.025124,-27.469771,POINT (152.823141 -27.603479),310031284
18438,9015,152.823141,-27.603479,153.025124,-27.469771,POINT (152.823141 -27.603479),310031284
18439,9464,153.074982,-27.397055,153.066000,-27.390000,POINT (153.074982 -27.397055),302031038
18440,9726,153.412197,-28.008783,153.400000,-28.016700,POINT (153.412197 -28.008783),309101268


In [63]:
all_postcode.to_csv("../data/curated/processed_postcode.csv", index=False)